In [4]:
import json
import torch
import faiss
import numpy as np
from transformers import AutoTokenizer, BertModel

# הגדרת המודל והטוקנייזר של heBERT
model_name = "avichr/heBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)



C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at avichr/heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight

In [14]:
# פונקציה ליצירת embeddings
def embeddingVectors(text):
    print(text)
    encoding = tokenizer.batch_encode_plus(
        text,  # List of input texts
        padding=True,  # Pad to the maximum sequence length
        truncation=True,  # Truncate to the maximum sequence length if necessary
        return_tensors='pt',  # Return PyTorch tensors
        add_special_tokens=True  # Add special tokens CLS and SEP
    )
    input_ids = encoding['input_ids']  # Token IDs
    attention_mask = encoding['attention_mask']  # Attention mask

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        word_embeddings = outputs.last_hidden_state  # (batch_size, sequence_length, embedding_dim)
        # החזרת ה-embedding של ה-CLS token
        sentence_embedding = word_embeddings[:, 0, :]  # בחירת האיבר הראשון בכל רצף (CLS token)
    return sentence_embedding  # This contains the embeddings



In [6]:
# קריאת הקובץ JSON
import json
with open('sample_data/פרי גני שוע.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# שליפת כל השאלות מהקובץ
questions = []
for book in data['contain'].values():
    for chapter in book.values():
        for verse in chapter.values():
            for qa in verse:
                questions.append(qa['question'])

print(len(questions))

5158


In [7]:
import pickle
import os

def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

def load_from_pickle(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        print(f"Data loaded from {filename}")
        return data
    else:
        print(f"File {filename} not found!")
        return None

In [8]:
# # צייני את הנתיב לקובץ שברצונך להוריד
# files.download("sample_data/embeddings_step1heBERT.pkl")
# print(f"עיבוד ושמירת חלק {i+1} הסתיים בהצלחה.")
print(i)

NameError: name 'i' is not defined

In [9]:


import numpy as np
import os
# from google.colab import files
batch_size = 100
your_embedding_dimension = 768  # או כל מספר אחר בהתאם למודל שלך

def get_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

embeddings_filename = "save_embded/embeddings_step1heBERT.pkl"

# טען את הנתונים השמורים, אם קיימים
embeddings = load_from_pickle(embeddings_filename)
if embeddings is None:
    embeddings = np.array([]).reshape(0, your_embedding_dimension)  # וודא שהגדרת את your_embedding_dimension
else:
    print("נתונים שמורים נטענו בהצלחה.")

# קביעת נקודת התחלה לעיבוד
start_index = len(embeddings)
print(start_index)
for i, batch in enumerate(get_batches(questions, batch_size)):
    if i * batch_size < start_index:
        continue  # דלג על חלקים שכבר חושבו ושמורים

    # המרת כל השאלות ל-embeddings
    batch_embeddings = embeddingVectors(questions).numpy()

   # שמור את החלק הנוכחי ל-embeddings2 ושמור לקובץ
    embeddings = np.vstack([embeddings, batch_embeddings])
    save_to_pickle(embeddings, embeddings_filename)


    

Data loaded from save_embded/embeddings_step1heBERT.pkl
נתונים שמורים נטענו בהצלחה.
123792


In [9]:
print(i)

51


In [10]:
import faiss

# טען את ה-embeddings ששמרת

embeddings = load_from_pickle("save_embded/embeddings_step1heBERT.pkl")

# שלב נוסף - בניית FAISS index
index_filename = "save_embded/index_step21heBERT.pkl"

index = load_from_pickle(index_filename)
if index is None:
    # יצירת האינדקס אם הוא לא נטען מקובץ
    dimension2 = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension2)
    index.add(embeddings)
    print(index)
    save_to_pickle(index, index_filename)

Data loaded from save_embded/embeddings_step1heBERT.pkl
Data loaded from save_embded/index_step21heBERT.pkl


In [16]:


# פונקציה למציאת השאלות הקרובות ביותר
def find_nearest_questions(query, k=5):
    query_embedding = embeddingVectors([query]).cpu().numpy()
    print(1)
    print(index)
    distances, indices = index.search(query_embedding, k)
    nearest_questions = [questions[idx] for idx in indices[0]]
    return nearest_questions



In [12]:
# פונקציה לרנקר (הדוגמה היא פשוטה. ניתן לשפר בהתאם לצורך)
def rerank_documents1(query, documents, top_n=5):
    # המרת השאלה ל-embedding
    query_embedding = embeddingVectors([query]).numpy()

    # המרת המסמכים ל-embeddings
    doc_embeddings = embeddingVectors(documents).numpy()

    # חישוב המרחקים
    distances = np.linalg.norm(doc_embeddings - query_embedding, axis=1)

    # מציאת האינדקסים של המסמכים הקרובים ביותר
    nearest_indices = distances.argsort()[:top_n]

    # החזרת המסמכים המסודרים לפי הקרבה
    reranked_documents = [documents[idx] for idx in nearest_indices]
    return reranked_documents

def rerank_documents(query, docs, top_n):
    results = co.rerank(model="rerank-multilingual-v3.0", query=query, documents=docs, top_n=top_n, return_documents=True)
    reranked_docs = [res.document.text for res in results.results]
    return reranked_docs




In [17]:
# שאלה לדוגמה
query = "האם נשים חיבות בתפילה"

# מציאת השאלות הקרובות ביותר
nearest_questions = find_nearest_questions(query, k=10)

# רנקר על התוצאות הקרובות ביותר
reranked_questions = rerank_documents(query, nearest_questions, top_n=5)

print("השאלות הקרובות ביותר לשאלה שלך הן:")
for q in reranked_questions:
    print(q)

['האם נשים חיבות בתפילה']
1
<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001EE2ABB71B0> >


: 